<a href="https://colab.research.google.com/github/dqniell/kaplan_meier/blob/main/kaplan_meier_blank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lifelines


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 4.8 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=8f2c9d4d71b89a458df2871c7e9764f6a39987ffd7365bd8741e2aeb908673ba
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


## Kaplan-Meier Analysis for COMPAS Recidivism Predictions

Objectives:

1. Compare survival probabilities (recidivism rates) by COMPAS risk score levels (Low, Medium, High).
2. Explore recidivism rates by gender and race.
3. Analyze survival curves to observe disparities between groups over a two-year observation period.

We are using the COMPAS risk scores as a measure to evaluate recidivism predictions. The Kaplan-Meier analysis allows us to estimate the probability of "surviving" (not reoffending) over time and comparing survival rates between different subgroups.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Exercise 1: Connect to the Database and Explore Tables

Connect to the `compas.db` SQLite database and examine its structure. Our goal is to identify which tables contain relevant data for Kaplan-Meier analysis.

1. **Connect** to the database file.
2. **List all tables** in the database to see the structure.
3. **Preview the contents** of each table to understand the available fields.

In [ ]:
import sqlite3

# TODO: Connect to the database
conn = sqlite3.connect('/content/drive/MyDrive/compas.db')

# List all tables in the database
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)
print("Tables in database:")

# TODO: Print out the tables to see their names
print(tables)

Tables in database:
            name
0     casearrest
1  prisonhistory
2    jailhistory
3         compas
4        summary
5         charge
6         people


In [ ]:
# Define a function to preview the first few rows of each table
def preview_table(table_name, conn, limit=5):
    query = f"SELECT * FROM {table_name} LIMIT {limit};"
    return pd.read_sql(query, conn)

# Preview the structure and contents of each table
table_previews = {table: preview_table(table, conn) for table in tables['name']}

# TODO: Print out the table_previews to visualize their contents
print(table_previews)

{'casearrest':    id              name    case_number arrest_id                 arrest_date  \
0   1  aajah herrington  13005832CF10A   3026755  2013-04-23 00:00:00.000000   
1   2  aajah herrington  13005832CF10A   3026756  2013-04-23 00:00:00.000000   
2   3  aajah herrington  14007863MM10A   3144392  2014-05-13 00:00:00.000000   
3   4      aaliyah lovo  13015312CF10A   3084831  2013-11-03 00:00:00.000000   
4   5      aaliyah lovo  13015312CF10A   3084832  2013-11-03 00:00:00.000000   

  charge_degree  days_since_compas_arrest  person_id  
0          (F3)                         1        533  
1          (M2)                         1        533  
2          (M1)                      -384        533  
3          (F2)                         1        900  
4          (M1)                         1        900  , 'prisonhistory':    id  name    first middle       last                         dob  \
0   1  None    bilal   None   williams  1955-05-12 00:00:00.000000   
1   2  None  edu

## Exercise 2: Extract Relevant Data for Kaplan-Meier Analysis

In order to successfully extract the necessary data, we need to join information from multiple tables to gather:

1. **COMPAS Scores** from the `compas` table (`person_id`, `decile_score`, `score_text` (risk level), `screening_date`).
2. **Demographics** from the `people` table (`sex`, `race`, `r_offense_date`).
3. **Incarceration Periods** from the `jailhistory` table (`in_custody`, `out_custody`) to filter out times when individuals were incarcerated.

*Instructions*:

*   Use SQL queries to join these tables on `person_id`
*   Extract relevant fields and load the result into a pandas DataFrame.

*Note*: Create a `two_year_recid` column by checking if `r_offense_date` is within two years (730 days) from screening_date.

*   Set `start` to 0 (representing release date)
*   Set end based on whether recidivism occurred within two years:
    - If `two_year_recid is`, set `end` to the days until `r_offense_date`.
    - If `two_year_recid` is 0, set `end` to 730 days.





In [ ]:
# Define SQL query to extract and join relevant data
# Fill in the relevant data for each table (after table_name. )
query = """
SELECT
    compas.person_id ,
    compas.decile_score ,
    compas.score_text AS risk_level,
    compas.screening_date ,
    people.sex ,
    people.race ,
    people.r_offense_date ,
    jailhistory.in_custody ,
    jailhistory.out_custody
FROM
    compas
LEFT JOIN
    people ON compas.person_id = people.id
LEFT JOIN
    jailhistory ON people.id = jailhistory.person_id
"""

# Load data into a DataFrame
df = pd.read_sql(query, conn)


# Convert dates to datetime format (screening_date, r_offense_date, in_custody, out_custody)
df['screening_date'] = pd.to_datetime(df['screening_date'])
df['r_offense_date'] = pd.to_datetime(df['r_offense_date'])
df['in_custody'] = pd.to_datetime(df['in_custody'])
df['out_custody'] = pd.to_datetime(df['out_custody'])


# Calculate two_year_recid based on whether recidivism occurred within 2 years
# We'll create a new column 'two_year_recid': 1 for recidivated within 2 years, otherwise 0
# First check if 'r_offense_date' exists for each row of the data set, then check whether the time between
# 'screening_date' and 'r_offense_date' is 730 days or less
df['two_year_recid'] = (df['r_offense_date'] - df['screening_date']).dt.days <= 730


# Set start time to 0 (create a 'start' column)
df['start'] = 0

# Define end time based on recidivism status (create an 'end' column)
if df['two_year_recid']:
    df['end'] = (df['r_offense_date'] - df['screening_date']).dt.days
else:
    df['end'] = 730



# Drop rows with missing essential data: subset of 'screening_date' and 'end'
df.dropna(subset=['screening_date', 'end'], inplace=True)

# Sort the DataFrame by 'screening_date'
df.sort_values(by='screening_date', inplace=True)

# Display the first few rows of the prepared DataFrame
df.head()


<ipython-input-36-9882ffba803a>:44: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if df['two_year_recid'].bool():


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## Exercise 3: Kaplan-Meier Analysis for COMPAS Risk Levels

1. Use `lifelines` to fit a Kaplan-Meier model for survival analysis.
2. Plot the survival curves for different risk levels (`Low`, `Medium`, `High`) to observe the recidivism trends over time.

*Note*: Remove rows with missing or "N/A" values in the risk level column.

In [ ]:
# Import KaplanMeierFitter from lifelines
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt

# Initialize Kaplan-Meier fitter
kmf = KaplanMeierFitter()

# Filter out rows with missing or "N/A" risk levels (score_text) and limit 'end' to be within 0 to 730 days (.notna() might be helpful!)


# Initialize the plot
plt.figure(figsize=(10, 6))

# Plot Kaplan-Meier curves for each risk level (score_text) fill in *** placeholders
kmf = KaplanMeierFitter()
for level in df_filtered['***'].unique():
    subset = df_filtered[df_filtered['***'] == level]
    kmf.fit(durations=subset['***'], event_observed=subset['***'], label=f'***: {level}')
    kmf.plot()

# Customize the plot
plt.title("***")
plt.xlabel("***")
plt.ylabel("***")
plt.xlim(***)  # Set x-axis limit to 0-730
plt.xticks(range(***))  # Show ticks from 0 to 730 in intervals of 100 days

# Display the plot
plt.show()

SyntaxError: invalid syntax (<ipython-input-32-99930da35897>, line 25)

## Exercise 4: Kaplan-Meier Analysis by Demographic Groups (Sex)

1. Compare Kaplan-Meier curves across sex.
2. Compare recidivism rates for female and male individuals across different COMPAS risk levels (Low, Medium, High). This allows us to see how recidivism probabilities differ by both gender and risk classification.

Helpful Tips:
*   Use two side-by-side subplots to display the curves for the two racial groups separately.
*   Ensure both plots span from 0 to 730 days, with x-axis tick marks every 100 days.
*   Clearly label each plot with the corresponding racial group, and include legends to show the COMPAS risk levels.

In [ ]:
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

# Fill in the *** placeholders

# Initialize Kaplan-Meier fitter


# Initialize subplots for side-by-side display
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Define COMPAS risk levels to iterate over
risk_levels = ['***', '***', '***']
colors = ['***', '***', '***']  # Colors for each risk level

# Plot for Female
female_subset = df[df['***'] == '***']
for risk, color in zip(risk_levels, colors):
    level_data = female_subset[female_subset['***'] == risk] # score_text
    kmf.fit(durations=level_data['***'], event_observed=level_data['***'], label=f'***: {risk}')
    kmf.plot(ax=axes[0], color=color)
axes[0].set_title("***")
axes[0].set_xlabel("***")
axes[0].set_ylabel("***")
axes[0].set_xlim(***)  # Set x-axis limit to 0-730
axes[0].set_xticks(range(***))  # Show ticks from 0 to 730 in intervals of 100 days

# Plot for Male
male_subset = df[df['***'] == '***']
for risk, color in zip(risk_levels, colors):
    level_data = male_subset[male_subset['***'] == risk] # score_text
    kmf.fit(durations=level_data['***'], event_observed=level_data['***'], label=f'***: {risk}')
    kmf.plot(ax=axes[1], color=color)
axes[1].set_title("***")
axes[1].set_xlabel("***")
axes[1].set_ylabel("***")
axes[1].set_xlim(***)  # Set x-axis limit to 0-730
axes[1].set_xticks(range(***))  # Show ticks from 0 to 730 in intervals of 100 days

# Adjust layout and add a legend
axes[0].legend(title="***")
axes[1].legend(title="***")
plt.tight_layout()

# Display the plots
plt.show()


## Exercise 5: Kaplan-Meier Analysis by Demographic Groups (Race)

1. Compare Kaplan-Meier curves across race.
2. Compare recidivism rates for two different racial groups, each broken down by COMPAS risk levels (Low, Medium, High). This will help us examine how survival probabilities differ by both race and risk classification.

Helpful Tips:
*   Use two side-by-side subplots to display the curves for the two racial groups separately.
*   Ensure both plots span from 0 to 730 days, with x-axis tick marks every 100 days.
*   Clearly label each plot with the corresponding racial group, and include legends to show the COMPAS risk levels.

*Note*: Select any two races from the dataset (e.g., Caucasian, African-American, Hispanic, Asian, Other); race strings are case-sensitive. Feel free to try different combinations and observe any trends.

In [ ]:
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

# Initialize Kaplan-Meier fitter


# Initialize subplots for side-by-side display
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Define the two racial groups you want to compare (feel free to change these; CASE-SENSITIVE)
race1 = '***'
race2 = '***'

# Define COMPAS risk levels to iterate over
risk_levels = ['***', '***', '***']
colors = ['***', '***', '***']  # Colors for each risk level

# Plot for the first racial group
race1_subset = df[df['***'] == race1]
for risk, color in zip(risk_levels, colors):
    level_data = race1_subset[race1_subset['***'] == risk] # score_text
    kmf.fit(durations=level_data['***'], event_observed=level_data['***'], label=f'***: {risk}')
    kmf.plot(ax=axes[0], color=color)
axes[0].set_title(f"***")
axes[0].set_xlabel("***")
axes[0].set_ylabel("***")
axes[0].set_xlim(***)  # Set x-axis limit to 0-730
axes[0].set_xticks(range(***))  # Show ticks from 0 to 730 in intervals of 100 days

# Plot for the second racial group
race2_subset = df[df['***'] == race2]
for risk, color in zip(risk_levels, colors):
    level_data = race2_subset[race2_subset['***'] == risk] # score_text
    kmf.fit(durations=level_data['***'], event_observed=level_data['***'], label=f'***: {risk}')
    kmf.plot(ax=axes[1], color=color)
axes[1].set_title(f"***")
axes[1].set_xlabel("***")
axes[1].set_ylabel("***")
axes[1].set_xlim(***)  # Set x-axis limit to 0-730
axes[1].set_xticks(range(***))  # Show ticks from 0 to 730 in intervals of 100 days

# Adjust layout and add a legend
axes[0].legend(title="***")
axes[1].legend(title="***")
plt.tight_layout()

# Display the plots
plt.show()



## Exercise 5: Calculate Median Survival Times

**Objective**: Calculate the median survival time, which indicates the time at which 50% of the population has recidivated.

1. First, calculate the median survival time for the entire dataset. This gives a general benchmark of when half of the population has recidivated.
2. Then, calculate the median survival time for each COMPAS risk level (e.g., Low, Medium, High). This allows us to see if there are significant differences in the timing or recidivism between risk groups.

*Note*: Filter out entries with 'N/A' in the `score_text` column if needed.

In [ ]:
from lifelines import KaplanMeierFitter

# Fill in the *** placeholders

# Filter out entries with 'N/A' in the 'score_text' column
df = df[df['***'] != '***']

# Initialize the Kaplan-Meier fitter


# Calculate median survival time for the entire dataset
kmf.fit(durations=df['***'], event_observed=df['***'])
median_survival = kmf.median_survival_time_

# Print out the median survival times


# Median survival times by risk level
for level in df['***'].unique():  # Adjust if 'score_text' is the correct column for risk level
    subset = df[df['***'] == level]
    kmf.fit(durations=subset['***'], event_observed=subset['***'])
    median_survival = kmf.median_survival_time_

    # Print these out to visualize the differences (use fstrings!)


## Exercise 6: Interpret the Results

**Objective**: Reflect on and interpret the Kaplan-Meier analysis results:
1. How does the recidivism rate change across different COMPAS risk levels?
    - Observe the median survival times for each risk level. Does the median survival time decrease as the risk level increases (i.e., High risk has a shorter median survival time than Low risk)?

2. Are there any noticeable differences in recidivism rates among gender or racial groups?
    - Compare the Kaplan-Meier curves and survival probabilities for different genders and racial groups from earlier exercises. Do certain groups recidivate faster than others?


1. High-Risk Group: Median survival time is 272 days, which indicates that half of the individuals in this group reoffend within about 9 months after release. This suggests a high recidivism rate among high-risk individuals.

  Medium-Risk Group: Median survival time is 431 days, meaning this group takes longer to reach the 50% recidivism mark compared to the high-risk group, which is expected.

  Low-Risk Group: Median survival time is "infinity," implying that fewer than 50% of low-risk individuals reoffend within the two-year observation period. This is consistent with their lower risk classification.

2. Gender: The survival probabilities likely reveal that men recidivate at a higher rate and earlier time points compared to women. Men in the high-risk group may show a faster decline in survival probability, suggesting they are more prone to reoffend. For women, even in the high-risk group, the survival probability may decrease at a slower rate than men.

  Race: If we compare different racial groups (e.g., Caucasian and African-American), the curves may indicate disparities. For instance, African-American individuals may exhibit a higher rate of recidivism in similar risk categories compared to Caucasian individuals, potentially highlighting racial disparities. This could be due to a range of socio-economic factors and historical biases that may influence both the likelihood of reoffending and the assessment itself.

In [ ]:
# Cool plot that displays every race in this dataset
# Initialize the plot
plt.figure(figsize=(10, 6))

# Plot Kaplan-Meier curves for each race
for race in df['race'].unique():
    subset = df[df['race'] == race]
    kmf.fit(durations=subset['end'], event_observed=subset['two_year_recid'], label=f'Race: {race}')
    kmf.plot()

# Customize the plot
plt.title("Kaplan-Meier Curve by Race (0 to 730 Days)")
plt.xlabel("Days since Screening")
plt.ylabel("Survival Probability (No Recidivism)")
plt.xlim(0, 730)  # Set x-axis limit to 0-730
plt.xticks(range(0, 731, 100))  # Show ticks from 0 to 730 in intervals of 100 days

# Display the plot
plt.show()
